# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import xgboost as xgb

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV, 
    StratifiedKFold, 
    cross_validate,
)

from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    make_scorer,
    precision_recall_curve,
)

import warnings
from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.auxiliares_ml import downcast_dataframe, testar_modelos_com_undersampling

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

### Pré-processamento: 

- Utilizaremos o **PowerTransformer** devido ao grande número de outliers, **exceto em 'Time'**. <br>
- Em **'Time'** utilizaremos o MinMaxScaler.

In [4]:
# Criando os escaladores

power_transformer = PowerTransformer(method='yeo-johnson')  # Funciona para valores positivos e negativos
minmax_scaler = MinMaxScaler()

# Aplicando a normalização nas colunas específicas

transacoes['Amount'] = power_transformer.fit_transform(transacoes[['Amount']])
transacoes['Time'] = minmax_scaler.fit_transform(transacoes[['Time']])

transacoes.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.119013,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-1.128464,0


In [5]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

### XGBoost 

In [6]:
# Configuração do cross-validation

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Criando o modelo XGBoost

modelo_xgb = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

In [7]:
# Métricas personalizadas

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}

In [8]:
# Executa o cross-validation

cv_results = cross_validate(modelo_xgb, X, y, cv=cv, scoring=scoring, return_train_score=False)

In [9]:
# Exibir os resultados

print("Resultados por dobra:")
for i in range(3):
    print(f"\nDobra {i+1}:")
    print(f"  Acurácia:  {cv_results['test_accuracy'][i]:.4f}")
    print(f"  Precisão:  {cv_results['test_precision'][i]:.4f}")
    print(f"  Recall:    {cv_results['test_recall'][i]:.4f}")

Resultados por dobra:

Dobra 1:
  Acurácia:  0.9996
  Precisão:  0.9236
  Recall:    0.8110

Dobra 2:
  Acurácia:  0.9996
  Precisão:  0.9247
  Recall:    0.8232

Dobra 3:
  Acurácia:  0.9996
  Precisão:  0.9178
  Recall:    0.8171


### Utilizando o GridSearchCV para melhorar os parâmetros do XGBoost

In [10]:
# Definição do modelo
modelo_xgb = xgb.XGBClassifier(random_state=0, scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1]))

# Definição dos hiperparâmetros a serem testados
param_grid = {
    'max_depth': [6, 9, 12],            # Testa árvores menos profundas e mais profundas
    'n_estimators': [100, 200, 300],    # Testa diferentes quantidades de árvores
    'learning_rate': [0.1, 0.2, 0.3],   # Testa diferentes taxas de aprendizado
    'gamma': [0, 0.1, 0.2],             # Testa regularização da complexidade da árvore
    'subsample': [0.8, 1.0],            # Testa redução de amostras para cada árvore
    'colsample_bytree': [0.8, 1.0]      # Testa redução de features usadas por árvore
}

In [11]:
# Configuração do GridSearchCV

grid_search = GridSearchCV(
    estimator=modelo_xgb,
    param_grid=param_grid,
    scoring='recall',  # Foco principal em Recall para evitar fraudes não detectadas
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=0),
    n_jobs=-1,  # Usa todos os núcleos disponíveis
    verbose=2
)

In [12]:
# Executa a busca pelos melhores parâmetros

grid_search.fit(X, y)

# Exibe os melhores parâmetros encontrados

print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)

# Melhor resultado encontrado

print("Melhor Recall encontrado:", grid_search.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
Melhores parâmetros encontrados:
{'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}
Melhor Recall encontrado: 0.839430894308943


### XGBoost (Melhores parâmetros)

In [13]:
# Configuração do cross-validation

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Criando o modelo XGBoost

modelo_xgb = xgb.XGBClassifier(
    max_depth=6,
    n_estimators=100,
    learning_rate=0.1,
    random_state=0,
    colsample_bytree=0.8,
    gamma=0, 
    subsample=0.8,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

In [14]:
# Métricas personalizadas

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}

In [15]:
# Executa o cross-validation

cv_results = cross_validate(modelo_xgb, X, y, cv=cv, scoring=scoring, return_train_score=False)

In [16]:
# Exibir os resultados

print("Resultados por dobra:")
for i in range(3):
    print(f"\nDobra {i+1}:")
    print(f"  Acurácia:  {cv_results['test_accuracy'][i]:.4f}")
    print(f"  Precisão:  {cv_results['test_precision'][i]:.4f}")
    print(f"  Recall:    {cv_results['test_recall'][i]:.4f}")

Resultados por dobra:

Dobra 1:
  Acurácia:  0.9994
  Precisão:  0.8084
  Recall:    0.8232

Dobra 2:
  Acurácia:  0.9995
  Precisão:  0.8519
  Recall:    0.8415

Dobra 3:
  Acurácia:  0.9994
  Precisão:  0.8000
  Recall:    0.8537


### OBSERVAÇÃO:

Agora utilizamos apenas o algoritmo que se saiu melhor até aqui: **XGBoost**

### NOTA:

Utilizamos o **GridSearchCV** para trazer novos parâmetros e encontrar os melhores valores possíveis para o **XGBoost**